<a href="https://colab.research.google.com/github/CARRIEE-WU/AIProject/blob/master/AIproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/a945120/AIProject.git

fatal: destination path 'AIProject' already exists and is not an empty directory.


In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def image_loader(image_name, imsize):
  loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])
  image = image.open(image_name)

  image = loader(image).unsqueeze(0)
  return image.to(device, torch.float)

In [ ]:
def imshow(tensor, title=None):
  unloader = transforms.ToPILImage()
  image = tensor.cpu().clone()
  image = image.squeeze(0)
  image = unloader(image)
  plt.imshow(image)
  if title is not None:
    plt.title(title)
  plt.pause(0.001)

In [ ]:
imsize = 512 if torch.cuda.is_available() else 128
image_directory = "../images/"
style_img = image_loader(image_directory + "meadow_painting.jpg", imsize)
content_img = image_loader(image_directory + "love_1.jpg", imsize)
assert style_img.size() == content_img.size(), "we need to import style and content images of the same size"
plt.ion()

In [ ]:
plt.figure()
imshow(style_img, title='Style Image')

In [ ]:
plt.figure()
imshow(content_img, title='Content Image')

In [ ]:
class ContentLoss(nn.Module):
  def __init__(self, target):
    super(ContentLoss, self).__init__()

    self.target = target.detach()

  def forward(self, input):
    self.loss = F.mse_loss(input, self.target)
    return input

In [ ]:
def gram_matrix(input):
  a,b,c,d = input.size()
  features = input.view(a * b, c * d)
  G = torch.mm(features, features.t())
  return G.div(a * b * c * d)

In [ ]:
class StyleLoss(nn.Module):
